In [1]:
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 3.2 MB/s eta 0:00:0000:0100:01


In [1]:
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port = 9042)

In [2]:
session = cluster.connect()

In [3]:
cluster.metadata.keyspaces

{'system_auth': <cassandra.metadata.KeyspaceMetadata at 0x119870d30>,
 'system_schema': <cassandra.metadata.KeyspaceMetadata at 0x1198dda30>,
 'dungeons': <cassandra.metadata.KeyspaceMetadata at 0x11cf80be0>,
 'system_distributed': <cassandra.metadata.KeyspaceMetadata at 0x11cf825e0>,
 'system': <cassandra.metadata.KeyspaceMetadata at 0x11cf884f0>,
 'system_traces': <cassandra.metadata.KeyspaceMetadata at 0x11cf966a0>,
 'system_views': <cassandra.metadata.KeyspaceMetadata at 0x11cf96e50>,
 'system_virtual_schema': <cassandra.metadata.KeyspaceMetadata at 0x11cfc0670>}

In [4]:
session.set_keyspace("dungeons")

In [35]:
print([e for e in session.execute('''DESCRIBE TABLES;''')])

[Row(keyspace_name='dungeons', type='table', name='hall_of_fame'), Row(keyspace_name='dungeons', type='table', name='top_horde'), Row(keyspace_name='dungeons', type='table', name='user_stats')]


## Test the query

In [33]:
import json
import pandas as pd

def sortFn(dict):
  return dict['n_killed']


def topk_players(country, event_id, k):
    query =f'''SELECT email, user_name, COUNT(*) as n_killed from top_horde WHERE country='{country}' AND event_id={event_id} GROUP BY country, event_id, email;'''
    raw = [{'email': row.email, 'user_name': row.user_name, 'n_killed': row.n_killed} for row in session.execute(query)]
    sorted_data = sorted(raw, key=lambda x: x['n_killed'], reverse=True)
    top_k = json.dumps(sorted_data[:k], indent=4)
    df = pd.DataFrame(sorted_data[:k])
    positions = [str(i)+'º' for i in range(1, k+1)]
    df.insert(0, 'Position', positions)
    df.set_index('Position', inplace=True)
    return top_k, df

result, df = topk_players('de_DE', 0, 7)
df
    

    

,email,user_name,n_killed
Position,,,
1º,xullrich@example.org,hartmanngloria,19
2º,silvesterconradi@example.net,bernhardmangold,15
3º,gretelholsten@example.com,johannekruschwitz,14
4º,pero13@example.net,zdravko34,14
5º,roehrichtpaulina@example.org,oderwaldgabriella,14
6º,scholtzanatolij@example.com,natalija00,14
7º,stiffelfreia@example.net,preissrudolf,14


In [31]:
import json



def leaderboard_by_country(country):
    final = []
    for i in range(20):
        to_insert = {}
        
        query = f"SELECT dungeon_name, email, user_name, time_minutes, date from hall_of_fame WHERE country='{country}' AND dungeon_id={i} LIMIT 5;"
        aux = session.execute(query)
        dungeon_id = i
        res = [{'dungeon_name':row.dungeon_name, 'email': row.email, 'user_name': row.user_name,
        'time_minutes': row.time_minutes, 'date': str(row.date)} for row in aux]
        dungeon_name = res[0]['dungeon_name']
        to_insert['dungeon_id'] = i
        to_insert['dungeon_name'] = dungeon_name
        to_insert['Top_5'] = []
        for e in res:
            del e['dungeon_name']
            to_insert['Top_5'].append(e)
            
        final.append(to_insert)
    final_json = json.dumps(final, indent=4)
    headers = ['dungeon_id', 'dungeon_name', 'email', 'user_name', 'time_minutes', 'date']
    array_for_df = []
    for e in final:
        for i in e['Top_5']:
            array_for_df.append([e['dungeon_id'], e['dungeon_name'], i['email'], i['user_name'], i['time_minutes'], i['date']])
    df = pd.DataFrame(array_for_df, columns=headers)
    df.set_index(['dungeon_id', 'dungeon_name'], inplace=True)
    return final_json, df
        

final, df = leaderboard_by_country('es_ES')
df

email  \
dungeon_id dungeon_name                                                             
0          Burghap, Prison of the Jealous Hippies           macario68@example.com   
           Burghap, Prison of the Jealous Hippies          begonavera@example.net   
           Burghap, Prison of the Jealous Hippies            cbarrena@example.net   
           Burghap, Prison of the Jealous Hippies             vcasals@example.org   
           Burghap, Prison of the Jealous Hippies            haydee45@example.net   
...                                                                           ...   
19         Dalhylles, Culverts of the Grumpy Bandits  corneliocarreno@example.org   
           Dalhylles, Culverts of the Grumpy Bandits     rouraoctavia@example.net   
           Dalhylles, Culverts of the Grumpy Bandits     tejadatamara@example.com   
           Dalhylles, Culverts of the Grumpy Bandits    fidelapozuelo@example.com   
           Dalhylles, Culverts of the Grumpy Bandits     anibalvalero@example.net   

                                                         user_name  \
dungeon_id dungeon_name                                              
0          Burghap, Prison of the Jealous Hippies     julianesther   
           Burghap, Prison of the Jealous Hippies      victorino97   
           Burghap, Prison of the Jealous Hippies        cecilia28   
           Burghap, Prison of the Jealous Hippies         nestor81   
           Burghap, Prison of the Jealous Hippies          julie30   
...                                                            ...   
19         Dalhylles, Culverts of the Grumpy Bandits     ruperta73   
           Dalhylles, Culverts of the Grumpy Bandits       nacio44   
           Dalhylles, Culverts of the Grumpy Bandits       icarpio   
           Dalhylles, Culverts of the Grumpy Bandits    fernanda49   
           Dalhylles, Culverts of the Grumpy Bandits    ritacuenca   

                                                      time_minutes  \
dungeon_id dungeon_name                                              
0          Burghap, Prison of the Jealous Hippies                0   
           Burghap, Prison of the Jealous Hippies                0   
           Burghap, Prison of the Jealous Hippies                0   
           Burghap, Prison of the Jealous Hippies                1   
           Burghap, Prison of the Jealous Hippies                1   
...                                                            ...   
19         Dalhylles, Culverts of the Grumpy Bandits             0   
           Dalhylles, Culverts of the Grumpy Bandits             0   
           Dalhylles, Culverts of the Grumpy Bandits             0   
           Dalhylles, Culverts of the Grumpy Bandits             0   
           Dalhylles, Culverts of the Grumpy Bandits             0   

                                                                     date  
dungeon_id dungeon_name                                                    
0          Burghap, Prison of the Jealous Hippies     2022-03-25 05:52:29  
           Burghap, Prison of the Jealous Hippies     2019-12-17 03:26:00  
           Burghap, Prison of the Jealous Hippies     2019-10-15 15:11:14  
           Burghap, Prison of the Jealous Hippies     2022-11-25 03:04:31  
           Burghap, Prison of the Jealous Hippies     2022-10-07 23:42:16  
...                                                                   ...  
19         Dalhylles, Culverts of the Grumpy Bandits  2022-12-27 17:50:10  
           Dalhylles, Culverts of the Grumpy Bandits  2022-12-26 12:17:03  
           Dalhylles, Culverts of the Grumpy Bandits  2022-12-20 14:21:34  
           Dalhylles, Culverts of the Grumpy Bandits  2022-12-17 05:12:03  
           Dalhylles, Culverts of the Grumpy Bandits  2022-12-15 13:37:09  

[100 rows x 4 columns]

In [38]:
def user_stats_per_dun(email, dungeon_id):
    final = []
    query = f"select time_minutes, date from user_stats where email='{email}' AND dungeon_id={dungeon_id};"
    aux = session.execute(query)
    for row in aux:
        final.append({'time_minutes':row.time_minutes, 'date':str(row.date)})
    
    final_json = json.dumps(final, indent=4)
    df = pd.DataFrame(final)
    return final_json, df
result, df = user_stats_per_dun('macario68@example.com',0)
df

,time_minutes,date
0,17,2022-09-18 18:48:40
1,34,2022-06-24 01:07:16
2,19,2022-06-11 15:11:25
3,27,2022-05-18 20:01:22
4,0,2022-03-25 05:52:29
5,20,2022-03-16 12:45:05
6,40,2022-03-06 02:01:56
7,29,2021-11-09 05:14:13
8,19,2021-10-11 10:31:03
9,46,2021-10-10 01:41:04
